In [2]:
import pandas as pd

In [3]:
df_eleitorados = pd.read_csv("eleitorado/perfil_eleitorado_2020/perfil_eleitorado_2020.csv", encoding="Latin 1")

In [4]:
df_resultados = pd.read_csv("resultados/SP_turno_1.csv", encoding="Latin 1", on_bad_lines='skip')

In [10]:
print(df_eleitorados)
print(df_resultados)

        DT_GERACAO;"HH_GERACAO";"ANO_ELEICAO";"SG_UF";"CD_MUNICIPIO";"NM_MUNICIPIO";"CD_MUN_SIT_BIOMETRIA";"DS_MUN_SIT_BIOMETRIA";"NR_ZONA";"CD_GENERO";"DS_GENERO";"CD_ESTADO_CIVIL";"DS_ESTADO_CIVIL";"CD_FAIXA_ETARIA";"DS_FAIXA_ETARIA";"CD_GRAU_ESCOLARIDADE";"DS_GRAU_ESCOLARIDADE";"QT_ELEITORES_PERFIL";"QT_ELEITORES_BIOMETRIA";"QT_ELEITORES_DEFICIENCIA";"QT_ELEITORES_INC_NM_SOCIAL"
0        01/08/2020;"08:37:48";"2020";"SP";"70734";"SÃO...                                                                                                                                                                                                                                                                                                                                      
1        01/08/2020;"08:37:48";"2020";"SP";"70734";"SÃO...                                                                                                                                                                            